In [17]:
using DataFrames, MLDataUtils
using Clustering, Distances
using CSV
using Random
using Logging
using Printf

# Set up Logging - we recommend to use this command to avoid package warnings during the model training process.
logger = Logging.SimpleLogger(stderr, Logging.Warn);
global_logger(logger);

In [2]:
#### Set parameters for the learners
cr = :silhouette # VALIDATION criterion totrain ICOT
method = "ICOT_local"
warm_start = :oct; 
# in the paper,  awarmstart slution to intilazation algorithm, oct use kmeas labels as 
# as a uspervised algorithm. greedy option fits a cart tree to lebels? try? or not good as kmenas
geom_search = true 
# s a boolean parameter that controls where the algorithm will enable the geometric component of the feature space search. See details in Section 3.3.1 of the paper.
threshold = 0.99 # the percentile of gaps for each 
seed = 1
gridsearch = true
num_tree_restarts = 20
# is an integer specifying the number of random restarts to use in the local search algorithm. Must be positive and defaults to 100. The performance of the tree typically increases as this value is increased, but with quickly diminishing returns. The computational cost of training increases linearly with this value.
complexity_c = 0.0
min_bucket = 100
maxdepth = 5 # can change ! use grid search for tuning!

###### Step 1: Prepare the data
# Read the data - recommend the use of the (deprecated) readtable() command to avoid potential version conflicts with the CSV package.
dataset = readtable("dataset_R.csv"); 
# println(dataset[1, :])


┌ Warning: readtable is deprecated, use CSV.read from the CSV package instead
│   caller = top-level scope at In[2]:17
└ @ Core In[2]:17


In [21]:
# data_x = select(dataset, Not(:label))
# data
#seperate x and y
true_label = dataset[:, end]
data_x = dataset[:, 1:end-1]
# Prepare data for ICOT: features are 
# stored in the matrix X, and the warm-start labels are stored in y
X = data_full[:, 1:end-2];
y = data_full[:, :true_labels];

# train model 

data_array = convert(Matrix{Float64}, data_x);
# Get the number of observations and features
n, p = size(data_array)
data_t = data_array';

##### Step 2: Fit K-means clustering on the dataset to generate a warm-start for ICOT
#Fix the seed
Random.seed!(seed);

# The ruspini dataset has pre-defined clusters, which we will use to select the cluster count (K) for the K-means algorithm. 
# In an unsupervised setting (with no prior-known K), the number of clusters for K means can be selected using the elbow method.
K = 10

# Run k-means and save the assignments 
kmeans_result = kmeans(data_t, K);
assignment = kmeans_result.assignments;

# concat data and feed into next model
data_full = DataFrame(hcat(dataset, assignment, makeunique=true));

#rename column
if !hasproperty(data_full, :true_labels)
    rename!(data_full, :label => :true_labels)
elseif !hasproperty(data_full, :kmean_assign)
    rename!(data_full, :x1 => :kmean_assign)
else
    println("all done")
end

,android_permission_CAMERA,android_permission_READ_CONTACTS,android_permission_WRITE_CONTACTS
,Float64⍰,Float64⍰,Float64⍰
1,-0.436297,1.79316,-0.4029
2,-0.436297,1.79316,-0.4029
3,-0.436297,1.79316,-0.4029
4,-0.436297,1.79316,-0.4029
5,-0.436297,1.79316,-0.4029
6,-0.436297,1.79316,-0.4029
7,-0.436297,1.79316,-0.4029
8,-0.436297,1.79316,-0.4029
9,-0.436297,1.79316,-0.4029


In [23]:


# Run ICOT with an OCT warm-start: fit an OCT as a supervised learning problem with labels "y" and use this as the warm-start
function icot(maxdepth)
    warm_start = :oct
    lnr_ws_oct = ICOT.InterpretableCluster(ls_num_tree_restarts=num_tree_restarts, ls_random_seed=seed, cp=complexity_c, max_depth=maxdepth,
        minbucket=min_bucket, criterion=cr, ls_warmstart_criterion=cr, kmeans_warmstart=warm_start,
        geom_search=geom_search, geom_threshold=threshold)
    run_time_icot_ls_oct = @elapsed ICOT.fit!(lnr_ws_oct, X, y)

    score_ws_oct = ICOT.score(lnr_ws_oct, X, y, criterion=:dunnindex)
    score_al_ws_oct = ICOT.score(lnr_ws_oct, X, y, criterion=:silhouette)
    ICOT.showinbrowser(lnr_ws_oct)

    @printf("dunnindex = %.4f\n", float(score_ws_oct))
    @printf("silhouette = %.4f\n", float(score_al_ws_oct))

end


icot (generic function with 1 method)

In [ ]:
# grid search for max max_depth
# unknown cannot use for gridsearch
# for maxdepth in [4, 5, 6, 7, 8, 9, 10]
#     @time begin
#     @printf("MAXDEPTH = %1i\n", maxdepth)
#     icot(maxdepth)
#     end
# end

In [26]:
# @printf("MAXDEPTH = %1i\n", maxdepth)
icot(3)

┌ Warning: implicit `dims=2` argument now has to be passed explicitly to specify that distances between columns should be computed
│   caller = ip:0x0
└ @ Core :-1
┌ Warning: This copy of Interpretable AI software is for academic purposes only and not for commercial use.
└ @ IAIBase /home/iai/.julia/packages/IAIBase/pOrUV/src/precompile.jl:19
Training trees...100%|██████████████████████████████████| Time: 0:00:29
Training trees...100%|██████████████████████████████████| Time: 0:00:31
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = dunn_score(::Array{Float64,2}, ::Array{Int64,1}) at clustering_tree.jl:132
└ @ ICOT /home/iai/.julia/packages/ICOT/34UmY/src/clustering/clustering_tree.jl:132
Training trees...100%|██████████████████████████████████| Time: 0:00:22
Training trees...100%|██████████████████████████████████| Time: 0:00:31
Training trees...  1%|▍                                 |  ETA: 0:05:06:08

InterruptException: [91mInterruptException:[39m